In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [4]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [5]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [49]:
DLL_Injection = spark.sql(
'''
SELECT 
    b.module_loaded,
    a.process_granted_access,
    b.process_name,
    a.process_id,
    b.process_id
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND (a.process_granted_access & 40) == 40 -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
    AND b.process_name = a.process_target_name
JOIN sysmon_events c
    ON b.process_name = c.process_target_name
    AND c.event_id = 8
    AND c.thread_start_function = "LoadLibraryA"
WHERE
     b.event_id = 7
'''
).show(100,False)

+--------------------------------------------------------------------------------------------------------------------------+----------------------+------------+----------+----------+
|module_loaded                                                                                                             |process_granted_access|process_name|process_id|process_id|
+--------------------------------------------------------------------------------------------------------------------------+----------------------+------------+----------+----------+
|c:\windows\system32\ole32.dll                                                                                             |2047999               |notepad.exe |5452      |3124      |
|c:\windows\system32\mscoree.dll                                                                                           |2047999               |notepad.exe |5452      |3124      |
|c:\windows\microsoft.net\framework64\v4.0.30319\mscoreei.dll                        

In [20]:
AND (process_granted_access & 5184) == 5184 -- 5184 is decimal for 0x1440. The minimal privileges you need to access process handle
    AND a.process_granted_access = "2047999"

SyntaxError: invalid syntax (<ipython-input-20-29f0fe263280>, line 1)

In [26]:
2047999 & 40 == 40

True